In [ ]:
(-math.tanh(s*(sim+cutoff))+1)/2

In [ ]:
import math
import numpy as np
def reward_function(params):

    waypoints=params["waypoints"]

    #hyperparameter
    
    # look_mode = 'lookbehind'

    #scales how close the bot follow the centre line, > more restrict, < less
    cos_sim_scaler = 5

    #proximity to track, proportion of track in inches
    threshold=12.5/15

    #half track width is 15in = 0.381m
    #car width is 5in = 0.127m
    #0.5width-0.5carwidth = 12.5in = 31.75cm = 0.3175m
    half_track_width = 0.381

    #scale how sharp the proximity cutoff, > more, < less
    cliff_scaler = 14 

    max_speed =3.1


    def lookskip(current_waypoint,step):
        # idk precisely why it works, it just works
        #retrive the waypoint by step from the closest waypoint(the second one)
        if step > 0:
            index = abs(current_waypoint[1] + step) % len(waypoints)
            
        else:
            index = -abs(current_waypoint[0] + step) % len(waypoints)
    
        return np.array(waypoints[index])
    
    def cosine_similarity(v, w): 
        return np.dot(v,w)/(np.linalg.norm(v)*np.linalg.norm(w)+1e-5)
        
    def quadrant(theta):
        if theta<0: 
            theta=theta+2*math.pi
        return [math.cos(theta), math.sin(theta)]
        
    def score(x,s):
        if x<0:
            return 1e-5
        else:
            return x**s
        
    def tracklimit_score(x):
        if x < threshold:
            return -((1.2*x)**cliff_scaler)+1
        else:
            return 0 

    theta = params["heading"]/180*math.pi

    car_dir_vect = np.array(quadrant(theta))

    # if look_mode == 'lookahead':
    #     closest_coords = lookskip(params["closest_waypoints"])
    # else:
    #     closest_coords = lookskip(params["closest_waypoints"])
        
    track_dir_vect = lookskip(params["closest_waypoints"],2) - lookskip(params["closest_waypoints"],-1)
    
    # track_dir_vect = closest_coords[1] - closest_coords[0]
    
    x = cosine_similarity(track_dir_vect, car_dir_vect)
    reward_score = score(x,cos_sim_scaler)

    #multiply reward score by proportion of max speed
   
    #add a cliff to drop off reward score once car has passed 12.5in from center. i.e. threshold is 12.5/15 of 1 side of the track 

    proportion_distance_from_center = params["distance_from_center"]/half_track_width
    
    reward_score = reward_score*tracklimit_score(proportion_distance_from_center)*(params["speed"]/max_speed)
    if params["is_offtrack"]:
        reward_score = 1e-5
    else:
        reward_score *= 1
    return reward_score

In [ ]:
def reward_function(params):
    waypoints = params['waypoints']

    #hyperparameter
    s = 6 # scales the tanh activation, s set of {0.,R} 

    cutoff = 0 #point where x=0 (transition point) for the point to go from + to - 

    def lookskip(current_waypoint,step):
        # idk precisely why it works, it just works
        #retrive the waypoint by step from the closest waypoint(the second one)

        last_waypoint_index = len(waypoints)-1

        diff =  current_waypoint[1]+step - last_waypoint_index

        return np.array(waypoints[diff-1])
    
    def cosine_similarity(v, w): 
        return np.dot(v,w)/(np.linalg.norm(v)*np.linalg.norm(w)+1e-5)



    coord1 = lookskip(params['closest_waypoints'],3)
    coord2 = lookskip(params['closest_waypoints'],5)
    coord3 =  lookskip(params['closest_waypoints'],7)


    #vec AB = B - A
    vec1 = coord1 - coord2
    vec2 = coord3 - coord2

    # if close to -1, opposite vectors(straight line)
    sim = cosine_similarity(vec1,vec2)

    if params['is_offtrack']:
        reward = 0
    else:
        if params["is_left_of_center"]:
            #if left, the cosine will get rewarded if it is close to -1(meaning straight line)
            #if direction wrong, will result in negative, can check graph e.g. -tanh(6(x+0.5))
            reward = -math.tanh(s(sim+cutoff))
        else:
            #if right, the cosine will get rewarded if it is close to 0(meaning curve line)
            reward = math.tanh(s(sim+cutoff))

    return reward

In [63]:
import numpy as np
waypoints = [(0,0),(1,1),(2,2),(3,3),(4,4),(5,5),(6,6),(7,7)]
current_waypoints = [1,2]
def lookskip(current_waypoint,step):
    # idk precisely why it works, it just works
    #retrive the waypoint by step from the closest waypoint(the second one)

    if step > 0:
        index = abs(current_waypoint[1] + step) % len(waypoints)
        
    else:
        index = -abs(current_waypoint[0] + step) % len(waypoints)
        


    return np.array(waypoints[index])

In [72]:
lookskip(current_waypoints,-6)

array([3, 3])

In [52]:
15%8

7